INITIALIZATION

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StringType, DateType
from pyspark.sql.functions import col, trim
from pyspark.sql.window import Window

READ BRONZE TABLE

In [0]:
df = spark.table("workspace.bronze.crm_prd_info")

In [0]:
#TRIMMING
for field in df.schema.fields:
    if isinstance(field.dataType, StringType):
        df = df.withColumn(field.name, trim(col(field.name)))

In [0]:
#PRODUCT KEY PARSING

df = df.withColumn("cat_id", F.regexp_replace(F.substring(col("prd_key"), 1, 5), "-", "_"))
df = df.withColumn("prd_key", F.substring(col("prd_key"), 7, F.length(col("prd_key"))))

In [0]:
#COST CLEANUP
df = df.withColumn("prd_cost", F.coalesce(col("prd_cost"), F.lit(0)))

In [0]:
#PRODUCTION LINE NORMALIZATION
df = (
    df
    # Normalize product line
    .withColumn(
        "prd_line",
        F.when(F.upper(col("prd_line")) == "M", "Mountain")
         .when(F.upper(col("prd_line")) == "R", "Road")
         .when(F.upper(col("prd_line")) == "S", "Other Sales")
         .when(F.upper(col("prd_line")) == "T", "Touring")
         .otherwise("n/a")
    )
)

In [0]:
#DATE CASTING
df = df.withColumn("prd_start_dt", col("prd_start_dt").cast(DateType()))

In [0]:
#RENAMING COLUMNS
RENAME_MAP = {
    "prd_id": "product_id",
    "cat_id": "category_id",
    "prd_key": "product_number",
    "prd_nm": "product_name",
    "prd_cost": "product_cost",
    "prd_line": "product_line",
    "prd_start_dt": "start_date",
    "prd_end_dt": "end_date"
}
for old_name, new_name in RENAME_MAP.items():
    df = df.withColumnRenamed(old_name, new_name)

In [0]:
#Sanity checks of dataframe
df.limit(10).display()
     

WRITING SILVER TABLE

In [0]:
df.write.mode("overwrite").format("delta").saveAsTable("workspace.silver.crm_products")

In [0]:

%sql
SELECT * FROM workspace.silver.crm_products LIMIT 10